In [ ]:
# Requirements
%pip install requests

In [ ]:
# Import libraries
import requests
import json

In [ ]:
# Function to perform NER using LLM
def perform_ner(text):
    prompt = f"""
    Perform Named Entity Recognition (NER) on the text below. Follow these rules STRICTLY:
    1. Identify only these entity types: PRODUCT, BRAND, COLOR, SIZE, MATERIAL, STYLE.
    2. Output MUST be valid JSON format matching this exact schema:
    ```json
    [
        {{"entity": "entity_name", "type": "ENTITY_TYPE"}}
    ]
    3. Do NOT include any explanations, notes, or non-JSON text.
    4. Do NOT return the given text to perform NER on, give only the JSON output.

    5.If no entities exist, return empty list [].

    6.Text: "{text}"

    7.DO NOT PRINT ANYTHING OTHER THAN OUTPUT JSON:"""

    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "deepseek-r1:14b",
            # "model": "llama2:13b",
            "prompt": prompt,
            "stream": False,
            "temperature": 0.5,
            "frequency_penalty": 0.5,
        }
    )
    result = response.json()
    print("Ollama API response:", result)
    # Ollama returns {'response': '...'}; extract and parse JSON from output
    output = result['response']
    try:
        # Try to extract the JSON array from the output
        entities = json.loads(output[output.find('['):output.rfind(']')+1])
    except Exception as e:
        entities = []
    return entities

In [ ]:
# Sample text to perform NER on
text = """
Description & fit
H&M Premium SelectionNew Arrival
Loose-fit shirt in an airy, woven linen and viscose blend. Collar, buttons at front, dropped shoulders, and long sleeves with button at cuffs. Rounded hem, longer at back.
"""

# print(perform_ner(text))
print("Model reply:", perform_ner(text))


Ollama API response: {'model': 'deepseek-r1:14b', 'created_at': '2025-05-13T09:03:10.556660932Z', 'response': '<think>\nAlright, let\'s tackle this NER task step by step. The user provided a text snippet about an H&M shirt and specified certain entity types to identify: PRODUCT, BRAND, COLOR, SIZE, MATERIAL, STYLE. They also emphasized strict adherence to the output format.\n\nFirst, I\'ll read through the text carefully. The mention of "H&M" is clear; that\'s definitely a BRAND. Next, looking for PRODUCT—I see it mentions "Loose-fit shirt," which fits under PRODUCT. For MATERIAL, the description says "woven linen and viscose blend," so both materials are identified there.\n\nI don\'t see any indications of COLOR in the text, just descriptions like \'airy\' but that\'s more of an adjective than a color name. SIZE isn\'t mentioned either; it\'s described as loose-fit, which relates to style rather than specific sizes. STYLE is covered with terms like "Loose-fit," "longer at back," and s